<a href="https://colab.research.google.com/github/aman5319/Focal-Loss/blob/master/FocalLoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Focal Loss

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

## Cross Entropy Loss

For Understanding Focal Loss we need to understand Cross Entropy Loss

In pytorch Cross Entropy Loss = LogSoftmax + NLLLoss

NLLLoss is negative log likelihood which is used in multiclass classifier.

$$
\boldsymbol{\mathcal{L}}=-\frac{1}{n}\sum_{i=1}^{n}\log(\hat{y}^{(i)})
$$


#### Below is the code to understand NLL Loss https://isaacchanghau.github.io/post/loss_functions/

In [2]:
def NLLLoss(logs, targets):
    out = torch.zeros_like(targets, dtype=torch.float)
    for i in range(len(targets)):
        out[i] = logs[i][targets[i]]
    return -out.mean()

x = torch.randn(3, 5)
y = torch.LongTensor([4, 1, 2])
cross_entropy_loss = torch.nn.CrossEntropyLoss()
log_softmax = torch.nn.LogSoftmax(dim=1)
x_log = log_softmax(x)

nll_loss = torch.nn.NLLLoss()
print("Torch CrossEntropyLoss: ", cross_entropy_loss(x, y))
print("Torch NLL loss: ", nll_loss(x_log, y))
print("Custom NLL loss: ", NLLLoss(x_log, y))
# Torch CrossEntropyLoss:  tensor(1.8739)
# Torch NLL loss:  tensor(1.8739)
# Custom NLL loss:  tensor(1.

Torch CrossEntropyLoss:  tensor(3.0131)
Torch NLL loss:  tensor(3.0131)
Custom NLL loss:  tensor(3.0131)


### Binary Cross Entropy
 $$
 \boldsymbol{\mathcal{L}} = -y*log(p) - (1-y)*log(1-p)  
 $$
 p is the probability and y is true label

## [Focal Loss](https://arxiv.org/pdf/1708.02002.pdf)

#### Binary Focal Loss

In [0]:
def binary_focal_loss(y_pred , y_true,gamma=2.0 , alpha=0.25 ,reduction="mean",function=torch.sigmoid,**kwargs):
    """
    Binary Version of Focal Loss
    :args
    
    y_pred : prediction
    
    y_true : true target labels
    
    gamma: dampeing factor default value 2 works well according to reasearch paper
    
    alpha : postive to negative ratio default value 0.25 means 1 positive and 3 negative can be tuple ,list ,int and float
    
    reduction = mean,sum,none

    function = can be sigmoid or softmax or None
    
    **kwargs: parameters to pass in activation function like dim in softmax
    
    """
    if isinstance(alpha,(list,tuple)):
        pos_alpha = alpha[0] # postive sample ratio in the entire dataset
        neg_alpha = alpha[1] #(1-alpha) # negative ratio in the entire dataset
    elif isinstance(alpha ,(int,float)):
        pos_alpha = alpha
        neg_alpha = (1-alpha)
        
    # if else in function can be simplified be removing setting to default to sigmoid  for educational purpose
    if function is not None:
        y_pred = function(y_pred , **kwargs) #apply activation function
    else :
        assert ((y_pred <= 1) & (y_pred >= 0)).all().item() , "negative value in y_pred value should be in the range of 0 to 1 inclusive"
    
    pos_pt = torch.where(y_true==1 , y_pred , torch.ones_like(y_pred)) # positive pt (fill all the 0 place in y_true with 1 so (1-pt)=0 and log(pt)=0.0) where pt is 1
    neg_pt = torch.where(y_true==0 , y_pred , torch.zeros_like(y_pred)) # negative pt
    
    pos_modulating = (1-pos_pt)**gamma # compute postive modulating factor for correct classification the value approaches to zero
    neg_modulating = (neg_pt)**gamma # compute negative modulating factor
    
    
    pos = -pos_alpha* pos_modulating*torch.log(pos_pt) #pos part
    neg = -neg_alpha* neg_modulating*torch.log(1-neg_pt) # neg part
    
    loss = pos+neg  # this is final loss to be returned with some reduction
    
    # apply reduction
    if reduction =="mean":
        return loss.mean()
    elif reduction =="sum":
        return loss.sum()
    elif reduction =="none":
        return loss # reduction mean
    else:
        raise f"Wrong reduction {reduction} is choosen \n choose one among [mean,sum,none]  "
    

In [4]:
y_pred = torch.randn(32,2)
y_true = torch.empty(32, 2).random_(2)
F.binary_cross_entropy_with_logits(y_pred,y_true) , binary_focal_loss(y_pred,y_true,gamma=0,alpha=[1,1]) # to test the correctness of method

(tensor(0.7474), tensor(0.7474))

In [5]:
binary_focal_loss(y_pred,y_true) # to test the correctness of method

tensor(0.1416)

#### Categorical Focal Loss

This focal loss is written for only object detection is SSD and YOLO where we assume that 75% anchors are background and 25% of them are foreground
https://blog.zenggyu.com/en/post/2018-12-05/retinanet-explained-and-demystified/#fnref10

$$
L_{cls} = -\sum_{i=1}^{K}(y_ilog(p_i)(1-p_i)^\gamma \alpha_i + (1 - y_i)log(1 - p_i)p_i^\gamma (1 - \alpha_i))
$$

In [0]:
def focal_loss(y_pred,y_true,alpha=0.25,gamma=2,):
    
    if isinstance(alpha,(list,tuple)):
        fore_alpha = alpha[0] # postive sample ratio in the entire dataset
        back_alpha = alpha[1] #(1-alpha) # negative ratio in the entire dataset
    elif isinstance(alpha ,(int,float)):
        fore_alpha = alpha
        back_alpha = (1-alpha)
        
    n_positives = (y_true!=0).sum() # all postive anchors for 20 class
  
    y_true = torch.eye(y_pred.shape[-1])[y_true].cuda() # one hot vector for all prediction
    y_pred = F.softmax(y_pred,dim=1) # apply softmax
    
    # in the dataset background classes is taken in the front so 1 background class + 20 classes = 21 classes
    back_pred = y_pred[:,0:1] # 1st column background
    fore_pred = y_pred[:,1:]  # 20 columns foreground
    back_true = y_true[:,0:1]  # 1st column background
    fore_true = y_true[:,1:] # 20 columns foreground
    
    alpha_factor = torch.cat([ back_true *back_alpha ,  fore_true * fore_alpha],dim=1)  ## alpha factor 
    
    focal_weight = torch.cat([ back_true * back_pred , fore_true * (1-fore_pred) ] ,dim=1) #because background is also a class so (1-back_true) will lead to false output
    
    cross_entropy = -1 *  torch.log(y_pred) # normal cross entropy
    loss =  alpha_factor * (focal_weight ** gamma) * cross_entropy # focal loss with modulating factor
    
    # normalize the loss with positive anchors
    return loss.sum()/n_positives  # if want to use it for anything else other then SSD use loss = loss.sum()/len(y_pred)
    

In [19]:
# Generating SSD data with random configuration for testing
"""
confidence_prediction: (batch_size,num_anchors,num_classes) 
"""
confidence_prediction = torch.randn((24,8732,21))

"""
target: (batch_size,num_anchors,num_classes)
"""
target = torch.LongTensor(24,8732,).random_(21)

## Reshaping 
confidence_prediction = confidence_prediction.view(-1,21)
print(f"confidence_prediction shape {confidence_prediction.shape}")
target = target.view(-1)
target[:208800]=0
print(f"target shape {target.shape}")
confidence_prediction.requires_grad_(True)
confidence_prediction = confidence_prediction.cuda()
target = target.cuda()
(target!=0).sum()
F.cross_entropy(confidence_prediction , target) , focal_loss(confidence_prediction , target,alpha=[1,1],gamma=0)

confidence_prediction shape torch.Size([209568, 21])
target shape torch.Size([209568])


(tensor(3.5079, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor(3.5079, device='cuda:0', grad_fn=<DivBackward0>))

In [21]:
# Generating SSD data with random configuration for studying the code
"""
confidence_prediction: (batch_size,num_anchors,num_classes) 
"""
confidence_prediction = torch.randn((2,5,15))

"""
target: (batch_size,num_anchors,num_classes)
"""
target = torch.LongTensor(2,5,).random_(15)

## Reshaping 
confidence_prediction = confidence_prediction.view(-1,15)
print(f"confidence_prediction shape {confidence_prediction.shape}")
target = target.view(-1)
target[:6]=0
print(f"target shape {target.shape}")
confidence_prediction.requires_grad_(True)
confidence_prediction = confidence_prediction.cuda()
target = target.cuda()
print((target!=0).sum())
F.cross_entropy(confidence_prediction , target) , focal_loss(confidence_prediction , target)

confidence_prediction shape torch.Size([10, 15])
target shape torch.Size([10])
tensor(4, device='cuda:0')


(tensor(3.3223, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor(0.8725, device='cuda:0', grad_fn=<DivBackward0>))

### Note:- When Using Focal Loss for SSD in two ways loss function can be written

1. Using Sigmoid : If Sigmoid is used as a final activation layer then don't add any extra background class just use the number of classes with in the dataset. In this particular scenario binary cross entropy has be to used where 1 means foreground object and 0 means background object

2.  Using Softmax:- If Softmax is used as a final activation Layer then an extra background class is to be added so for pascal voc 2007 we will 20 classes with 1 extra background class we will have total 21 class. Use Cross Entropy to calculate the loss. we are using softmax
